Import Package

In [23]:
import io
import re
import os
import numpy as np
import pandas as pd

Load Data

In [24]:
with open('A_prot.txt', 'r') as txt:
    lines = [l for l in txt if l.startswith(' A')]
pre0 = pd.read_csv(io.StringIO(''.join(lines)), names=['0','1','2','3','4','5','6','7','8','9','10'], sep='\s+')
pre0.fillna('', inplace=True)

pre = pd.DataFrame(columns=['prot', 'code']) 
pre['prot'] = pre0['0']
pre['code'] = pre0['1'] + pre0['2'] + pre0['3'] + pre0['4'] + pre0['5'] + pre0['6'] + pre0['7'] + pre0['8'] + pre0['9'] + pre0['10']
pre

,prot,code
0,A*01:01:01:01,MAVMAPRTLLLLLSGALAL..TQTWAGSHSMRYFFTSVSRPGRGEP...
1,A*01:01:01:02N,-------------------..-------------------------...
2,A*01:01:01:03,-------------------..-------------------------...
3,A*01:01:01:04,-------------------..-------------------------...
4,A*01:01:01:05,-------------------..-------------------------...
...,...,...
31568,A*80:06,----K-K------------E-------P------------------...
31569,A*80:07,----K-K------------E-------P------------------...
31570,A*80:08N,
31571,A*80:09N,


Filter

In [25]:
filter = (pre0['0'] == 'A*01:01:01:01')
pre0[filter]

,0,1,2,3,4,5,6,7,8,9,10
0,A*01:01:01:01,MAVM,APRTLLLLLS,GALAL..TQT,WAGSHSMRYF,FTSVSRPGRG,EPRFIAVGYV,DDTQFVRFDS,DAASQKMEPR,APWIEQEGPE,YWDQETRNMK
7893,A*01:01:01:01,AHSQTDRANL,GTLRGYYNQS,EDGSHTIQIM,YGCDVGPDGR,FLRGY.RQDA,YDGKDY.IAL,NEDLRSWTAA,DMAAQITKRK,WEAVHAAE..,..........
15786,A*01:01:01:01,..QRRVYLEG,RCVDGLRRYL,EN...GKETL,QRTDPPKTHM,THHPISDHEA,TLRCWALGFY,PAEITLTWQR,DGED.QTQDT,ELVETRPAGD,GTFQKWAAVV
23679,A*01:01:01:01,VPSGEEQRYT,CHVQHEGLPK,PLTLRWELSS,QPTIPIVGII,AGLVLLGAVI,TGAVVAAVMW,RRKSSDRKGG,SYTQAASSDS,AQGSDVSLTA,CKV


Slice & Merge

In [26]:
pre1 = pre['code'][0:7893].reset_index()
pre2 = pre['code'][7893:15786].reset_index()
pre3 = pre['code'][15786:23679].reset_index()
pre4 = pre['code'][23679:31572].reset_index()

df = pd.DataFrame(columns=['prot', 'code']) 
df['prot'] = pre['prot'][0:7893]
df['code'] = df['code'].str.cat([pre1['code'], pre2['code'], pre3['code'], pre4['code']], sep='', na_rep='')
df['code'][0]

'MAVMAPRTLLLLLSGALAL..TQTWAGSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQKMEPRAPWIEQEGPEYWDQETRNMKAHSQTDRANLGTLRGYYNQSEDGSHTIQIMYGCDVGPDGRFLRGY.RQDAYDGKDY.IALNEDLRSWTAADMAAQITKRKWEAVHAAE..............QRRVYLEGRCVDGLRRYLEN...GKETLQRTDPPKTHMTHHPISDHEATLRCWALGFYPAEITLTWQRDGED.QTQDTELVETRPAGDGTFQKWAAVVVPSGEEQRYTCHVQHEGLPKPLTLRWELSSQPTIPIVGIIAGLVLLGAVITGAVVAAVMWRRKSSDRKGGSYTQAASSDSAQGSDVSLTACKV'

HW1

In [5]:
def HW1(field):
    str0 = []
    ref = df['code'][0]
    
    filt = (df['prot'] == field)
    index_field = df.index[filt].tolist()
    spec = df['code'][(index_field[0])]
    sub = re.compile('\*|[A-Z]')

    index_t = [substr.start() for substr in re.finditer(sub, spec)]
    
    str0 = ref[:len(spec)]
    for i in index_t:
        str0 = str0[:i] + spec[i] + str0[i+1:]
    
    str0 = str0.replace('.','')
    str0 = str0.replace('*','.')


    return str0

Input & Output

In [6]:
spec_field = input('field:')
print(HW1(spec_field))

.........................SHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQKMEPRAPWIEQERPEYWDQETRNVKAHSQTDRENLGTLRGYYNQSEAGSHTIQIMYGCDVGPDGRFLRGYRQDAYDGKDYIALNEDLRSWTAADMAAQITKRKWEAVHAAEQRRVYLEGRCVDGLRRYLENGKETLQRT...............................................................................................................................................................


HW2

In [39]:
def HW2(field):    
    filt = (df['prot'] == field)
    index_field = df.index[filt].tolist()
    spec = df['code'][(index_field[0])]
    sub = re.compile('\*|[A-Z]')

    index_t = [substr.start() for substr in re.finditer(sub, spec)]

    ts = ''
    for i in index_t:
        position = i - 25
        protein = spec[i]
        
        ts = f'{ts}, {position}:{protein}'
    ts = ts.strip(', ')
    return ts

In [40]:
HW2('A*01:02:01:01')

'9:S, 17:S'

Build Dataframe

In [48]:
df2 = pd.DataFrame(columns=['prot', 'code'])
df2['prot'] = [c for c in df['prot'] if c.startswith('A*01')]
df2['code'] = [HW2(c) for c in df2['prot']]
df2['code'][0] = ''

df2

,prot,code
0,A*01:01:01:01,
1,A*01:01:01:02N,"92:D, 93:P, 94:G, 95:P, 96:G, 97:R, 98:R, 99:S..."
2,A*01:01:01:03,
3,A*01:01:01:04,
4,A*01:01:01:05,
...,...,...
710,A*01:431,17:P
711,A*01:432,99:C
712,A*01:433N,277:X
713,A*01:434,108:P


Output

In [49]:
df2.to_csv('Group_HW2.txt',sep='\t',index=False)